In [ ]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
ncfile = Dataset('/p/work1/lloveras/moist_runs/rst_blob2days/wrfrst_d01_2021-01-03_00_00_00','r+')

In [ ]:
qv = ncfile.variables['QVAPOR'][0,:,:,:]
nz = qv.shape[0]
ny = qv.shape[1]
nx = qv.shape[2]

In [ ]:
### Roll to start
roll = 200
qv = np.roll(qv,roll,axis=2)

### Parameters
pert = np.zeros((nz,ny,nx))
dz = 0.2
dx = 20.
wave = False

### Location, decay scales, and wavelength
xnot = roll - 105 #180
ynot = 140
ax = 1000
ay = 1000
h = 1.0
l = 200

### Amplitude
amp = 0.15

for x in range(nx):
    for y in range(ny):
        for z in range(nz):
            r = np.sqrt(((x - xnot)*dx/ax)**2 + ((y - ynot)*dx/ay)**2)
            if r >= np.pi/2:
                r = np.pi/2.
                pert[z,y,x] = 0.
            elif wave:
                pert[z,y,x] = 2*amp*qv[z,y,x]*np.exp(-z*dz/h)*np.cos(-r)*np.sin(2*np.pi*x*dx/l)\
                                    *np.sin(2*np.pi*y*dx/l)
            else:    
                pert[z,y,x] = amp*qv[z,y,x]*np.exp(-z*dz/h)*np.cos(-r)
                
### Unroll
qv = np.roll(qv,-roll,axis=2)
pert = np.roll(pert,-roll,axis=2)

In [ ]:
ncfile.variables['QVAPOR'][0,:,:,:] = qv + pert

In [ ]:
ncfile.close()